In [ ]:
# %%
# Import
import pandas as pd
import sqlite3
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("ss_com8.db")
df0 = pd.read_sql_query("SELECT * from results", con)
# Verify that result of SQL query is stored in the dataframe
print(df0.head())
con.close()

# Normalization to currency format EUR/month
df0[['Prices','Currency format']] =df0['CENA'].str.split('  ',expand=True)
df0['Prices'] = df0['Prices'].str.replace(',','')
df0[["Prices"]] = df0[["Prices"]].apply(pd.to_numeric)
df0.loc[(df0['Currency format']=='€/ned.','Prices per week')]=df0['Prices']
df0.loc[(df0['Currency format']=='€/ned.','Prices')]=4*df0['Prices']
df0.loc[(df0['Currency format']=='€/dienā','Prices per day')]=df0['Prices']
df0.loc[(df0['Currency format']=='€/dienā','Prices')]=30*df0['Prices']
df0.loc[(df0['Currency format']=='€/mēn.','Prices per month')]=df0['Prices']
df0.loc[(df0['Currency format']=='€/mēn.','Prices')]=df0['Prices']
df0.loc[(df0['Currency format']=='€','Prices full')]=df0['Prices']
df0.loc[(df0['Currency format']=='€','Prices')]=df0['Prices']
df0 = df0.drop(['CENA'], axis=1)

# Normalization to land format m2
df0.loc[(df0['ESTATE_TYPE']=='plots-and-lands','LAND_M2')]=df0['PREMISE_M2']
df0.loc[(df0['ESTATE_TYPE']=='plots-and-lands','PREMISE_M2')]=' '
df0[['Land','Land format']]=df0['LAND_M2'].str.split(' ',expand=True)
df0.loc[(df0['Land']=='na','Land')]=''
df0.loc[((df0['Land format']=='m²')|(df0['Land format']==' '),'Land')]=df0['Land'].apply(pd.to_numeric)
df0.loc[(df0['Land format']=='ha.','Land')]=10000*df0["Land"].apply(pd.to_numeric)
df0 = df0.drop(['LAND_M2','Land format'], axis=1)

# Normalization of premise
df0['premise_m2']=df0['PREMISE_M2'].astype(str)
df0[['Premise','Premise format']]=df0['premise_m2'].str.split(' ',expand=True)
df0.loc[(df0['Premise']=='na','Premise')]=''
df0.loc[((df0['Premise format']=='m²')|(df0['Premise format']==''),'Premise')]=df0['Premise'].apply(pd.to_numeric)
df0.loc[(df0['Premise format']=='ha.','Premise')]=10000*df0["Premise"].apply(pd.to_numeric)
df0 = df0.drop(['PREMISE_M2','Premise format','premise_m2'], axis=1)

df0['Prices']=df0['Prices'].apply(pd.to_numeric)
df0['Premise']=df0['Premise'].apply(pd.to_numeric)
df0['Land']=df0['Land'].apply(pd.to_numeric)

df0.drop(df0[df0['Premise']<10].index, inplace=True)
df0.drop(df0[df0['Land']<df0['Premise']].index, inplace=True)

df0.loc[((df0['Prices']!='')&(df0['Premise']!=''),'Pricepremise')]=df0['Prices']/df0['Premise']
df0.loc[((df0['Prices']!='')&(df0['Land']!=''),'Priceland')]=df0['Prices']/df0['Land']

df0.drop(df0[df0['Priceland']>1000].index, inplace=True)
df0.drop(df0[(df0['ESTATE_TYPE'] == 'farms-estates')&(df0['Land']<5)].index, inplace=True)

# Normalization of STAVS
# flats - number of rooms
# homes-summer-residence, farms-estates - number of floors
# offices - floor  from all floors
# plots-and-lands - empty
df0[['STAVS 1','STAVS 2']] =df0['STAVS'].str.split('/',expand=True)
df0.loc[((df0['STAVS 1']=='na')|(df0['STAVS 1']=='Citi')|(df0['STAVS 1']=='-'),'STAVS 1')]=''
df0.loc[(df0['STAVS 1'].str.contains('.00', case=True, regex=True, na=False)),'STAVS 1']=df0['STAVS 1'].astype(str).str[0]
df0.loc[(df0['STAVS 2'].str.contains('.00', na=False)),'STAVS 2']=df0['STAVS 2'].astype(str).str[0]
df0.loc[((df0['ESTATE_TYPE']=='flats'),'Number of rooms')]=df0['STAVS 1']
df0.loc[((df0['ESTATE_TYPE']=='offices'),'Floor')]=df0['STAVS 1']
df0.loc[((df0['ESTATE_TYPE']=='offices'),'Max floor')]=df0['STAVS 2']
df0.loc[((df0['ESTATE_TYPE']=='homes-summer-residences')|(df0['ESTATE_TYPE']=='farms-estates'),'Number of floors')]=df0['STAVS 1']
df0 = df0.drop(['STAVS 1','STAVS 2','STAVS'], axis=1)

# Normalization of Location
df0[['1','2','3','4','5','6','7','8','9']] =df0['AD_LINK'].str.split('/',expand=True)

#Add Region and District
df0['Region']=df0['6']
df0.loc[(df0['AD_LINK'].str.count('/')==7,'District')]=df0['7']
df0.loc[(df0['AD_LINK'].str.count('/')==8,'District')]=df0['8']
df0.drop(df0[df0['Region']=='flats-abroad-latvia'].index, inplace = True)
df0 = df0.drop(['1','2','3','4','5','6','7','8','9','AD_LINK'], axis=1)

# Create categories according to a region
df0.loc[(df0['Region']=='riga','Region adj')]='riga'
df0.loc[(df0['Region']=='riga-region','Region adj')]='riga-region'
# cities are daugavpils, liepaja, jelgava, jurmala, ventspils, rezekne
df0.loc[((df0['District']=='daugavpils')|(df0['District']=='liepaja')|(df0['District']=='jelgava')|(df0['Region']=='jurmala')|(df0['District']=='ventspils')|(df0['District']=='rezekne'),'Region adj')]='cities'
df0.loc[((df0['Region']!='riga')&(df0['Region']!='riga-region')&(df0['District']!='daugavpils')&(df0['District']!='liepaja')&(df0['District']!='jelgava')&(df0['Region']!='jurmala')&(df0['District']!='ventspils')&(df0['District']!='rezekne'),'Region adj')]='other'

# Normalization of location (street)
df0['LOCATION'] = df0['LOCATION'].str.replace(', ','')
df0[['Town','Street and building']] =df0['LOCATION'].str.split('<br/>',expand=True)
# Can be changed according tp other datasets
df0['Street']=df0['Street and building']
df0 = df0.drop(['Town','LOCATION','Street and building'], axis=1)

# Normalization of TIME_STAMP
df0[['Date','Time']] =df0['TIME_STAMP'].str.split(' ',expand=True)
df0[['Year','Month','Day']] =df0['Date'].str.split('-',expand=True)
df0.loc[((df0['Month']=='01')|(df0['Month']=='02')|(df0['Month']=='03'),'Quarter')]='1'
df0.loc[((df0['Month']=='04')|(df0['Month']=='05')|(df0['Month']=='06'),'Quarter')]='2'
df0.loc[((df0['Month']=='07')|(df0['Month']=='08')|(df0['Month']=='09'),'Quarter')]='3'
df0.loc[((df0['Month']=='10')|(df0['Month']=='11')|(df0['Month']=='12'),'Quarter')]='4'
df0 = df0.drop(['Time','TIME_STAMP'], axis=1)

# Normalization of house type
df0.loc[((df0['HOUSE_TYPE']=='na')|(df0['HOUSE_TYPE']=='-'),'HOUSE_TYPE')]=''
df0.loc[(df0['HOUSE_TYPE']!='','HOUSE_TYPE')]=df0['HOUSE_TYPE']
df0['Price before discount']=df0['Prices']

# %%
# Import
import pandas as pd
import sqlite3
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("city24_20220804.db")
df_city0 = pd.read_sql_query("SELECT * from results", con)
# Verify that result of SQL query is stored in the dataframe
print(df_city0.head())
con.close()

df_city0['HOUSE_TYPE']=''
df_city0['Prices per week']=''

# Normalization of Location
df_city0[['L1','L2','L3','L4','L5','L6','L7','L8','L9']] =df_city0['AD_LINK'].str.split('/',expand=True)
df_city0.loc[(df_city0['L1']=='https:','Region')]=df_city0['L7']
df_city0.loc[(df_city0['L1']=='https:','Tr type')]=df_city0['L6']
df_city0.loc[(df_city0['L1']=='..','Region')]=df_city0['L8']
df_city0.loc[(df_city0['L1']=='..','Tr type')]=df_city0['L7']
df_city0.loc[(df_city0['L1']=='','Region')]=df_city0['L4']
df_city0.loc[(df_city0['L1']=='','Tr type')]=df_city0['L3']

df_city0.loc[((df_city0['Tr type']=='houses-for-sale')|(df_city0['Tr type']=='apartments-for-sale')|(df_city0['Tr type']=='pardod-dzivokli'),'TRANSACTION_TYPE')]='sell'
df_city0.loc[((df_city0['Tr type']=='houses-for-rent')|(df_city0['Tr type']=='apartments-for-rent')|(df_city0['Tr type']=='izire-dzivokli'),'TRANSACTION_TYPE')]='hand_over'
df_city0 = df_city0.drop(['L1','L2','L3','L4','L5','L6','L7','L8','L9','Tr type'], axis=1)

# Normalization of Region
# Create categories according to a region
df_city0[['Re1','Re2','Re3','Re4','Re5','Re6','Re7','Re8']] =df_city0['Region'].str.split('-',expand=True)
df_city0['Region adj']='other'
df_city0.loc[((df_city0['Re1']=='Riga')|(df_city0['Re1']=='riga'),'Region adj')]='riga'
df_city0.loc[((df_city0['Re1']=='Riga')|(df_city0['Re1']=='riga'),'District')]=df_city0['Re2']
# cities are daugavpils, liepaja, jelgava, jurmala, ventspils, rezekne
df_city0.loc[((df_city0['Re1']=='Daugavpils')|(df_city0['Re1']=='Liepaja')|(df_city0['Re1']=='Jelgava')|(df_city0['Re1']=='Jurmala')|(df_city0['Re1']=='Ventspils')|(df_city0['Re1']=='Rezekne'),'Region adj')]='cities'
df_city0.loc[((df_city0['Re1']=='Adazi')|(df_city0['Re1']=='Adazu')|(df_city0['Re1']=='Allazu')|(df_city0['Re1']=='Babite')|(df_city0['Re1']=='Babites')|(df_city0['Re1']=='Baldone')|(df_city0['Re1']=='Baldones')|(df_city0['Re1']=='Balozi')|(df_city0['Re1']=='Carnikava')|(df_city0['Re1']=='Carnikavas')|(df_city0['Re1']=='Daugmales')|(df_city0['Re1']=='Garkalne')|(df_city0['Re1']=='Garkalnes')|(df_city0['Re1']=='Incukalna')|(df_city0['Re1']=='Incukalns')|(df_city0['Re1']=='Kekava')|(df_city0['Re1']=='Kekavas')|(df_city0['Re1']=='Krimuldas')|(df_city0['Re1']=='malpils')|(df_city0['Re1']=='Malpils')|(df_city0['Re1']=='Marupe')|(df_city0['Re1']=='Marupes')|(df_city0['Re1']=='Mores')|(df_city0['Re1']=='Olaine')|(df_city0['Re1']=='Olaines')|(df_city0['Re1']=='Ropazu')|(df_city0['Re1']=='Salas')|(df_city0['Re1']=='Salaspils')|(df_city0['Re1']=='Saulkrasti')|(df_city0['Re1']=='Saulkrastu')|(df_city0['Re1']=='Seja')|(df_city0['Re1']=='Sejas')|(df_city0['Re1']=='Sigulda')|(df_city0['Re1']=='Siguldas')|(df_city0['Re1']=='Stopinu')|(df_city0['Re1']=='Vangazi'),'Region adj')]='riga-region'
df_city0 = df_city0.drop(['Re1','Re2','Re3','Re4','Re5','Re6','Re7','Re8'], axis=1)

df_city0.loc[((df_city0['ESTATE_TYPE']=='House')|(df_city0['ESTATE_TYPE']=='House for sale')|(df_city0['ESTATE_TYPE']=='House for rent'),'ESTATE_TYPE')]='homes-summer-residences'
df_city0.loc[((df_city0['ESTATE_TYPE']=='Appartment')|(df_city0['ESTATE_TYPE']=='Appartment for sale')|(df_city0['ESTATE_TYPE']=='Appartment for rent'),'ESTATE_TYPE')]='flats'

# Normalization of price
# Usual price
df_city0['CENA'] = df_city0['CENA'].str.strip()
df_city0.loc[((df_city0['CENA'].str.count('EUR')==1)|(df_city0['CENA'].str.count('€')==1),'One price')]=df_city0['CENA']
df_city0[['Ce1','Ce2','Ce3','Ce4']]=df_city0['One price'].str.split(' ',expand=True)
df_city0.loc[((df_city0['Ce2'].str.count('EUR')==1)|(df_city0['Ce2'].str.count('€')==1),'Prices')]=df_city0['Ce1']
df_city0.loc[((df_city0['Ce3'].str.count('EUR')==1)|(df_city0['Ce3'].str.count('€')==1),'Prices')]=df_city0['Ce1']+df_city0['Ce2']
df_city0.loc[((df_city0['Ce4'].str.count('EUR')==1)|(df_city0['Ce4'].str.count('€')==1),'Prices')]=df_city0['Ce1']+df_city0['Ce2']+df_city0['Ce3']
df_city0.loc[(((df_city0['CENA'].str.count('EUR')==1)|(df_city0['CENA'].str.count('€')==1))&(df_city0['TRANSACTION_TYPE']=='hand_over'),'Prices per month')]=df_city0['Prices']
df_city0.loc[(((df_city0['CENA'].str.count('EUR')==1)|(df_city0['CENA'].str.count('€')==1))&(df_city0['TRANSACTION_TYPE']=='hand_over'),'Currency format')]='€/mēn.'
df_city0.loc[(((df_city0['CENA'].str.count('EUR')==1)|(df_city0['CENA'].str.count('€')==1))&(df_city0['TRANSACTION_TYPE']=='sell'),'Prices full')]=df_city0['Prices']
df_city0.loc[(((df_city0['CENA'].str.count('EUR')==1)|(df_city0['CENA'].str.count('€')==1))&(df_city0['TRANSACTION_TYPE']=='sell'),'Currency format')]='€'

# Price per dayand per month in a one cell
df_city0.loc[(((df_city0['CENA'].str.count('day')==1)|(df_city0['CENA'].str.count('dienā')==1))&((df_city0['CENA'].str.count('month')==1)|(df_city0['CENA'].str.count('mēn')==1)),'Type price - day and month')]=df_city0['CENA']
df_city0['Bollean - Type price - day and month']=0
df_city0.loc[(((df_city0['CENA'].str.count('day')==1)|(df_city0['CENA'].str.count('dienā')==1))&((df_city0['CENA'].str.count('month')==1)|(df_city0['CENA'].str.count('mēn')==1)),'Bollean - Type price - day and month')]=1
df_city0[['Ct1','Ct2']]=df_city0['Type price - day and month'].str.split('/',expand=True)
df_city0[['Ct1.1','Ct1.2','Ct1.3','Ct1.4']]=df_city0['Ct1'].str.split(' ',expand=True)
df_city0.loc[((df_city0['Ct1.3']=='(day)')|(df_city0['Ct1.3']=='(dienā)'),'Prices per day')]=df_city0['Ct1.1']
df_city0.loc[((df_city0['CENA'].str.count('day')==1)&(df_city0['CENA'].str.count('month')==0),'Type price - day')]=df_city0['CENA']
df_city0[['Ct1.1','Ct1.2','Ct1.3']]=df_city0['Type price - day'].str.split(' ',expand=True)
df_city0.loc[((df_city0['Ct1.3']=='(day)')|(df_city0['Ct1.3']=='(dienā)'),'Prices per day')]=df_city0['Ct1.1']
df_city0[['Ct2.1','Ct2.2','Ct2.3','Ct2.4','Ct2.5']]=df_city0['Ct2'].str.split(' ',expand=True)
df_city0[['Ct2.4','Ct2.6']]=df_city0['Ct2.4'].str.split(')',expand=True)
df_city0[['Ct2.5','Ct2.6']]=df_city0['Ct2.5'].str.split(')',expand=True)
df_city0.loc[((df_city0['Ct2.5']=='(month')|(df_city0['Ct2.5']=='(mēn'),'Prices per month')]=df_city0['Ct2.2']+df_city0['Ct2.3']
df_city0.loc[((df_city0['Ct2.4']=='(month')|(df_city0['Ct2.4']=='(mēn'),'Prices per month')]=df_city0['Ct2.2']
df_city0 = df_city0.drop(['Ct1','Ct2','Ct1.1','Ct1.2','Ct1.3','Ct1.4','Ct2.1','Ct2.2','Ct2.3','Ct2.4','Ct2.5','Ct2.6','Type price - day'], axis=1)

df_city0['Prices per day'] = df_city0['Prices per day'].str.replace(' ','')
df_city0['Prices per day']=df_city0['Prices per day'].apply(pd.to_numeric)
df_city0.loc[((df_city0['Prices per day']>0),'Prices')]=df_city0['Prices per day']*30

df_city0_price_day_month=df_city0.copy()
df_city0_price_day_month.drop(df_city0_price_day_month[df_city0_price_day_month['Bollean - Type price - day and month']==0].index, inplace=True)
df_city0.drop(df_city0[df_city0['Bollean - Type price - day and month']==1].index, inplace=True)

df_city0_price_day=df_city0_price_day_month.copy()
df_city0_price_day['Prices per day']=df_city0_price_day['Prices per day'].apply(pd.to_numeric)
df_city0_price_day['Prices']=df_city0_price_day['Prices per day']*30
df_city0_price_day['Currency format']='€/dienā'
df_city0_price_day['Prices per month']=''

df_city0_price_month=df_city0_price_day_month.copy()
df_city0_price_month['Prices per month']=df_city0_price_month['Prices per month'].replace(' ','')
df_city0_price_month['Prices']=df_city0_price_month['Prices per month']
df_city0_price_month['Currency format']='€/mēn.'
df_city0_price_month['Prices per day']=''

df_city0.drop(df_city0[df_city0['Bollean - Type price - day and month']==1].index, inplace=True)
df_city0=pd.concat([df_city0,df_city0_price_day,df_city0_price_month], ignore_index=True)
df_city0['Prices']=df_city0['Prices'].replace(' ', '')

# Price after discount, disount, price before discount 
df_city0.loc[(((df_city0['CENA'].str.count('EUR')==3)|(df_city0['CENA'].str.count('€')==3)),'Type price - with discount')]=df_city0['CENA']
df_city0[['Cd1','Cd2','Cd3']]=df_city0['Type price - with discount'].str.split(' ',expand=True)
df_city0[['Cd1.1','Cd1.2','Cd1.3','Cd1.4','Cd1.5','Cd1.6','Cd1.7']]=df_city0['Cd1'].str.split(' ',expand=True)
df_city0[['Cd2.1','Cd2.2','Cd2.3','Cd2.4']]=df_city0['Cd2'].str.split(' ',expand=True)
df_city0[['Cd3.1','Cd3.2','Cd3.3','Cd3.4']]=df_city0['Cd3'].str.split(' ',expand=True)
df_city0.loc[((df_city0['Cd1.2'].str.count('EUR')==1)|(df_city0['Cd1.2'].str.count('€')==1),'Price after discount')]=df_city0['Cd1.1']
df_city0.loc[((df_city0['Cd1.3'].str.count('EUR')==1)|(df_city0['Cd1.2'].str.count('€')==1),'Price after discount')]=df_city0['Cd1.1']+df_city0['Cd1.2']
df_city0.loc[((df_city0['Cd1.4'].str.count('EUR')==1)|(df_city0['Cd1.4'].str.count('€')==1),'Price after discount')]=df_city0['Cd1.1']+df_city0['Cd1.2']+df_city0['Cd1.3']
df_city0.loc[((df_city0['Cd2.2'].str.count('EUR')==1)|(df_city0['Cd2.2'].str.count('€')==1),'Discount')]=df_city0['Cd2.1']
df_city0.loc[((df_city0['Cd2.3'].str.count('EUR')==1)|(df_city0['Cd2.3'].str.count('€')==1),'Discount')]=df_city0['Cd2.1']+df_city0['Cd2.2']
df_city0.loc[((df_city0['Cd2.4'].str.count('EUR')==1)|(df_city0['Cd2.4'].str.count('€')==1),'Discount')]=df_city0['Cd2.1']+df_city0['Cd2.2']+df_city0['Cd2.3']
df_city0.loc[((df_city0['Cd3.2'].str.count('EUR')==1)|(df_city0['Cd3.2'].str.count('€')==1),'Price before discount')]=df_city0['Cd3.1']
df_city0.loc[((df_city0['Cd3.3'].str.count('EUR')==1)|(df_city0['Cd3.3'].str.count('€')==1),'Price before discount')]=df_city0['Cd3.1']+df_city0['Cd3.2']
df_city0.loc[((df_city0['Cd3.4'].str.count('EUR')==1)|(df_city0['Cd3.4'].str.count('€')==1),'Price before discount')]=df_city0['Cd3.1']+df_city0['Cd3.2']+df_city0['Cd3.3']
df_city0.loc[(((df_city0['CENA'].str.count('EUR')==3)|(df_city0['CENA'].str.count('€')==3)),'Prices')]=df_city0['Price after discount']
df_city0.loc[(((df_city0['CENA'].str.count('EUR')==3)|(df_city0['CENA'].str.count('€')==3))&(df_city0['TRANSACTION_TYPE']=='hand_over'),'Prices per month')]=df_city0['Prices']
df_city0.loc[(((df_city0['CENA'].str.count('EUR')==3)|(df_city0['CENA'].str.count('€')==3))&(df_city0['TRANSACTION_TYPE']=='hand_over'),'Currency format')]='€/mēn.'
df_city0.loc[(((df_city0['CENA'].str.count('EUR')==3)|(df_city0['CENA'].str.count('€')==3))&(df_city0['TRANSACTION_TYPE']=='sell'),'Prices full')]=df_city0['Prices']
df_city0.loc[(((df_city0['CENA'].str.count('EUR')==3)|(df_city0['CENA'].str.count('€')==3))&(df_city0['TRANSACTION_TYPE']=='sell'),'Currency format')]='€'
df_city0 = df_city0.drop(['Cd1.1','Cd1.2','Cd1.3','Cd1.4','Cd1.5','Cd1.6','Cd1.7','Cd2.1','Cd2.2','Cd2.3','Cd3.1','Cd3.2','Cd3.3','Cd3.4','Cd1','Cd2','Cd3'], axis=1)

# Normalization to land format m2
df_city0.loc[(df_city0['LAND_M2']=='land?','LAND_M2')]=''
df_city0[['PREMISE_M2','LAND_M2']]=df_city0['PREMISE_M2'].str.split(', ',expand=True)
df_city0[['L1','L2','L3','L4']]=df_city0['LAND_M2'].str.split(' ',expand=True)
df_city0.loc[(df_city0['L1']=='Tontin','Land')]=df_city0['L3']
df_city0.loc[(df_city0['L1']=='Tontin','Land format')]=df_city0['L4']
df_city0.loc[(df_city0['L1']!='Tontin','Land')]=df_city0['L2']
df_city0.loc[(df_city0['L1']!='Tontin','Land format')]=df_city0['L3']
df_city0.loc[(df_city0['Land']=='na','Land')]=''
df_city0.loc[((df_city0['Land format']=='m²')|(df_city0['Land format']==' '),'Land')]=df_city0['Land'].apply(pd.to_numeric)

# Normalization of premise
df_city0['premise_m2']=df_city0['PREMISE_M2'].astype(str)
df_city0[['premise 1','premise 2','premise 3','premise 4']]=df_city0['premise_m2'].str.split(' ',expand=True)
df_city0.loc[(df_city0['premise 1']=='Tontin','Land')]=df_city0['premise 3']
df_city0.loc[(df_city0['premise 1']=='Tontin','Land format')]=df_city0['premise 4']
df_city0.loc[((df_city0['premise 1']=='Zeme')|(df_city0['premise 1']=='Plot'),'Land')]=df_city0['premise 2']
df_city0.loc[((df_city0['premise 1']=='Zeme')|(df_city0['premise 1']=='Plot'),'Land format')]=df_city0['premise 3']
df_city0.loc[((df_city0['premise 1']!='Zeme')&(df_city0['premise 1']!='Plot')&(df_city0['premise 1']!='Tontin'),'Premise')]=df_city0['premise 1']
df_city0.loc[((df_city0['premise 1']!='Zeme')&(df_city0['premise 1']!='Plot')&(df_city0['premise 1']!='Tontin'),'Premise format')]=df_city0['premise 2']
df_city0.loc[(df_city0['Premise']=='na','Premise')]=''
df_city0.loc[((df_city0['Premise format']=='m²')|(df_city0['Premise format']==''),'Premise')]=df_city0['Premise'].apply(pd.to_numeric)
df_city0.loc[(df_city0['Premise format']=='hectare.','Premise')]=10000*df_city0["Premise"].apply(pd.to_numeric)
df_city0 = df_city0.drop(['PREMISE_M2','Premise format','premise_m2','premise_m2','premise 1','premise 2','premise 3','premise 4'], axis=1)
df_city0.loc[(df_city0['Land format']=='hectare','Land')]=10000*df_city0["Land"].apply(pd.to_numeric)
df_city0 = df_city0.drop(['LAND_M2','Land format','L1','L2','L3','L4'], axis=1)

# Normalization of rooms and floors
df_city0[['R1','R2','R3']] =df_city0['ISTABAS'].str.split(' ',expand=True)
df_city0.loc[(df_city0['R3']=='ist.','Number of rooms')]=df_city0['R2']
df_city0.loc[(df_city0['R3']!='ist.','Number of rooms')]=df_city0['R1']
df_city0.loc[(df_city0['Number of rooms']=='na','Number of rooms')]=''
df_city0['Number of rooms']=df_city0['Number of rooms'].apply(pd.to_numeric)
df_city0 = df_city0.drop(['R1','R2','R3'], axis=1)

df_city0.loc[(df_city0['STAVS']=='na','STAVS')]=''
df_city0[['S1','S2','S3']] =df_city0['STAVS'].str.split(' ',expand=True)
df_city0[['S1','S2']] =df_city0['S1'].str.split(' ',expand=True)
df_city0[['S1','S4']] =df_city0['S1'].str.split('/',expand=True)
df_city0.loc[(df_city0['ESTATE_TYPE']=='homes-summer-residences','Number of floors')]=df_city0['S1']
df_city0.loc[(df_city0['ESTATE_TYPE']=='flats','Floor')]=df_city0['S1']
df_city0.loc[(df_city0['ESTATE_TYPE']=='flats','Max floor')]=df_city0['S4']
df_city0 = df_city0.drop(['S1','S2','S3','S4'], axis=1)

# Normalization of TIME_STAMP
df_city0[['Date','Time']] =df_city0['TIME_STAMP'].str.split(' ',expand=True)
df_city0[['Year','Month','Day']] =df_city0['Date'].str.split('-',expand=True)
df_city0.loc[((df_city0['Month']=='01')|(df_city0['Month']=='02')|(df_city0['Month']=='03'),'Quarter')]='1'
df_city0.loc[((df_city0['Month']=='04')|(df_city0['Month']=='05')|(df_city0['Month']=='06'),'Quarter')]='2'
df_city0.loc[((df_city0['Month']=='07')|(df_city0['Month']=='08')|(df_city0['Month']=='09'),'Quarter')]='3'
df_city0.loc[((df_city0['Month']=='10')|(df_city0['Month']=='11')|(df_city0['Month']=='12'),'Quarter')]='4'
df_city0 = df_city0.drop(['Time','TIME_STAMP'], axis=1)

#Different forms of presenting the name in the column "Street"
df_city0=df_city0.rename(columns={"APRAKSTS":"AD_TEXT", "ADRESE":"Street"})
df_city0['Premise']=df_city0['Premise'].apply(pd.to_numeric)
df_city0['Land']=df_city0['Land'].apply(pd.to_numeric)
df_city0['Prices']=df_city0['Prices'].apply(pd.to_numeric)
df_city0.loc[((df_city0['Prices']!='')&(df_city0['Premise']!=''),'Pricepremise')]=df_city0['Prices']/df_city0['Premise']
df_city0.loc[((df_city0['Prices']!='')&(df_city0['Land']!=''),'Priceland')]=df_city0['Prices']/df_city0['Land']

df_city0['Land']=df_city0['Land'].round(0)
df_city0['Premise']=df_city0['Premise'].round(0)
df_city0['Number of rooms']=df_city0['Number of rooms'].round(0)
df_city0['Floor']=df_city0['Floor'].apply(pd.to_numeric)
df_city0['Floor']=df_city0['Floor'].round(0)
df_city0['Max floor']=df_city0['Max floor'].apply(pd.to_numeric)
df_city0['Max floor']=df_city0['Max floor'].round(0)
df_city0['Number of floors']=df_city0['Number of floors'].apply(pd.to_numeric)
df_city0['Number of floors']=df_city0['Number of floors'].round(0)
#df_city0.loc[df_city0['Prices']>=100,'Prices round']=(df_city0['Prices']/10).round(0)
#df_city0.loc[df_city0['Prices']>=100,'Prices']=10*df_city0['Prices round']

df_city0.drop(df_city0[df_city0['Premise']<10].index, inplace=True)
df_city0.drop(df_city0[df_city0['Land']<df_city0['Premise']].index, inplace=True)
df_city0.drop(df_city0[df_city0['Priceland']>1000].index, inplace=True)

df_city=df_city0.copy()
df_city = df_city[['ID','AD_ID','AD_TEXT','HOUSE_TYPE','AD_SOURCE','ESTATE_TYPE','TRANSACTION_TYPE','Prices','Currency format','Prices per week','Prices per day','Prices per month','Prices full','Land','Premise','Pricepremise','Priceland','Number of rooms','Floor','Max floor','Number of floors','Region','District','Region adj','Street','Date','Year','Month','Day','Quarter','Price before discount']]

# %%
df=pd.concat([df0,df_city], ignore_index=True)

df['Prices']=df['Prices'].apply(pd.to_numeric)
df['Prices']=df['Prices'].round(2)
df['Pricepremise']=df['Pricepremise'].apply(pd.to_numeric)
df['Pricepremise']=df['Pricepremise'].round(2)
df['Priceland']=df['Priceland'].apply(pd.to_numeric)
df['Priceland']=df['Priceland'].round(2)
df['Prices full']=df['Prices full'].apply(pd.to_numeric)
df['Prices full']=df['Prices full'].round(2)
df['Prices per day']=df['Prices per day'].apply(pd.to_numeric)
df['Prices per day']=df['Prices per day'].round(2)
df['Prices per month']=df['Prices per month'].apply(pd.to_numeric)
df['Prices per month']=df['Prices per month'].round(2)

# Create new ID to this dataset and delete dublicates
df['Pr']=df['Prices'].astype(str)
df['Pr bd']=df['Price before discount'].astype(str)
df['Pre']=df['Premise'].astype(str)
df['St']=df['Street'].astype(str)
df['L']=df['Land'].astype(str)
df['M']=df['Month'].apply(pd.to_numeric)

df['ID_0']=df[['Pr bd','Pre','St','L']].apply(lambda x: '_'.join(x), axis=1)
df['Min date']=df.groupby('ID_0')['Date'].transform(min)
df['Max date']=df.groupby('ID_0')['Date'].transform(max)
df[['Date','Min date','Max date']] = df[['Date','Min date','Max date']].apply(pd.to_datetime)
df['Diff'] = (df['Date'] - df['Min date']).dt.days
df['Duration date']=(df['Max date']-df['Min date']).dt.days
df['N']=df['Diff']//90
df['N']=df['N'].astype(str)
df['ID_1']=df[['Pr bd','Pre','St','L','N']].apply(lambda x: '_'.join(x), axis=1)
df.drop_duplicates(subset=['ID_1','N'],keep = 'first',inplace=True)

df['ID_0']=df[['Pr','Pre','St','L']].apply(lambda x: '_'.join(x), axis=1)
df['Min date']=df.groupby('ID_0')['Date'].transform(min)
df['Max date']=df.groupby('ID_0')['Date'].transform(max)
df[['Date','Min date','Max date']] = df[['Date','Min date','Max date']].apply(pd.to_datetime)
df['Diff'] = (df['Date'] - df['Min date']).dt.days
df['Duration date']=(df['Max date']-df['Min date']).dt.days
df['N']=df['Diff']//90
df['N']=df['N'].astype(str)
df['ID_2']=df[['Pr','Pre','St','L','N']].apply(lambda x: '_'.join(x), axis=1)
df.drop_duplicates(subset=['ID_2','N'],keep = 'first',inplace=True)

df = df.drop(['Pr','Pr bd','Pre','St','L','M','N','ID_1','ID_0','Diff'], axis=1)

# %%
#Boolean types
#ESTATE_TYPE
df.loc[(df['ESTATE_TYPE']=='farms-estates','B-ESTATE_TYPE-farms-estates')]=1
df.loc[(df['ESTATE_TYPE']!='farms-estates','B-ESTATE_TYPE-farms-estates')]=0
df.loc[(df['ESTATE_TYPE']=='flats','B-ESTATE_TYPE-flats')]=1
df.loc[(df['ESTATE_TYPE']!='flats','B-ESTATE_TYPE-flats')]=0
df.loc[(df['ESTATE_TYPE']=='homes-summer-residences','B-ESTATE_TYPE-homes-summer-residences')]=1
df.loc[(df['ESTATE_TYPE']!='homes-summer-residences','B-ESTATE_TYPE-homes-summer-residences')]=0
df.loc[(df['ESTATE_TYPE']=='offices','B-ESTATE_TYPE-offices')]=1
df.loc[(df['ESTATE_TYPE']!='offices','B-ESTATE_TYPE-offices')]=0
df.loc[(df['ESTATE_TYPE']=='plots-and-lands','B-ESTATE_TYPE-plots-and-lands')]=1
df.loc[(df['ESTATE_TYPE']!='plots-and-lands','B-ESTATE_TYPE-plots-and-lands')]=0

#TRANSACTION_TYPE
df.loc[(df['TRANSACTION_TYPE']=='hand_over','B-TRANSACTION_TYPE-hand-over')]=1
df.loc[(df['TRANSACTION_TYPE']!='hand_over','B-TRANSACTION_TYPE-hand-over')]=0
df.loc[(df['TRANSACTION_TYPE']=='sell','B-TRANSACTION_TYPE-sell')]=1
df.loc[(df['TRANSACTION_TYPE']!='sell','B-TRANSACTION_TYPE-sell')]=0

#HOUSE_TYPE
df.loc[(df['HOUSE_TYPE']=='103.','B-HOUSE_TYPE-103.')]=1
df.loc[(df['HOUSE_TYPE']!='103.','B-HOUSE_TYPE-103.')]=0
df.loc[(df['HOUSE_TYPE']=='104.','B-HOUSE_TYPE-104.')]=1
df.loc[(df['HOUSE_TYPE']!='104.','B-HOUSE_TYPE-104.')]=0
df.loc[(df['HOUSE_TYPE']=='119.','B-HOUSE_TYPE-119.')]=1
df.loc[(df['HOUSE_TYPE']!='119.','B-HOUSE_TYPE-119.')]=0
df.loc[(df['HOUSE_TYPE']=='467.','B-HOUSE_TYPE-467.')]=1
df.loc[(df['HOUSE_TYPE']!='467.','B-HOUSE_TYPE-467.')]=0
df.loc[(df['HOUSE_TYPE']=='602.','B-HOUSE_TYPE-602.')]=1
df.loc[(df['HOUSE_TYPE']!='602.','B-HOUSE_TYPE-602.')]=0
df.loc[(df['HOUSE_TYPE']=='Čehu pr.','B-HOUSE_TYPE-Čehu pr.')]=1
df.loc[(df['HOUSE_TYPE']!='Čehu pr.','B-HOUSE_TYPE-Čehu pr.')]=0
df.loc[(df['HOUSE_TYPE']=='Hrušč.','B-HOUSE_TYPE-Hrušč.')]=1
df.loc[(df['HOUSE_TYPE']!='Hrušč.','B-HOUSE_TYPE-Hrušč.')]=0
df.loc[(df['HOUSE_TYPE']=='Jaun.','B-HOUSE_TYPE-Jaun.')]=1
df.loc[(df['HOUSE_TYPE']!='Jaun.','B-HOUSE_TYPE-Jaun.')]=0
df.loc[(df['HOUSE_TYPE']=='LT proj.','B-HOUSE_TYPE-LT proj.')]=1
df.loc[(df['HOUSE_TYPE']!='LT proj.','B-HOUSE_TYPE-LT proj.')]=0
df.loc[(df['HOUSE_TYPE']=='M. ģim.','B-HOUSE_TYPE-M. ģim.')]=1
df.loc[(df['HOUSE_TYPE']!='M. ģim.','B-HOUSE_TYPE-M. ģim.')]=0
df.loc[(df['HOUSE_TYPE']=='P. kara','B-HOUSE_TYPE-P. kara')]=1
df.loc[(df['HOUSE_TYPE']!='P. kara','B-HOUSE_TYPE-P. kara')]=0
df.loc[(df['HOUSE_TYPE']=='Priv. m.','B-HOUSE_TYPE-Priv. m.')]=1
df.loc[(df['HOUSE_TYPE']!='Priv. m.','B-HOUSE_TYPE-Priv. m.')]=0
df.loc[(df['HOUSE_TYPE']=='Renov.','B-HOUSE_TYPE-Renov.')]=1
df.loc[(df['HOUSE_TYPE']!='Renov.','B-HOUSE_TYPE-Renov.')]=0
df.loc[(df['HOUSE_TYPE']=='Specpr.','B-HOUSE_TYPE-Specpr.')]=1
df.loc[(df['HOUSE_TYPE']!='Specpr.','B-HOUSE_TYPE-Specpr.')]=0
df.loc[(df['HOUSE_TYPE']=='Staļina','B-HOUSE_TYPE-Staļina')]=1
df.loc[(df['HOUSE_TYPE']!='Staļina','B-HOUSE_TYPE-Staļina')]=0

# District types in Riga
df['B-District-riga-Āgenskalns']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='agenskalns')|(df['District']=='biekensala')|(df['District']=='tornjakalns')|(df['District']=='Agenskalns')|(df['District']=='Tornakalns')),'B-District-riga-Āgenskalns')]=1
df['B-District-riga-Boldērāja']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='bolderaya')|(df['District']=='daugavgriva')|(df['District']=='voleri')|(df['District']=='Bolderaja')|(df['District']=='Daugavgriva')),'B-District-riga-Boldērāja')]=1
df['B-District-riga-Centrs']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='centre')|(df['District']=='grizinkalns')|(df['District']=='kipsala')|(df['District']=='kliversala')|(df['District']=='stacija-tirgus')|(df['District']=='vecriga')|(df['District']=='Centrs')|(df['District']=='Grizinkalns')|(df['District']=='Kipsala')|(df['District']=='Kliversala')|(df['District']=='Vecriga')),'B-District-riga-Centrs')]=1
df['B-District-riga-CITS']=0
df.loc[((df['Region adj']=='riga')&(df['District']=='other'),'B-District-riga-CITS')]=1
df.loc[((df['Region adj']=='riga')&((df['District']=='Beberbeki')|(df['District']=='Darzini')|(df['District']=='Pardaugava')|(df['District']=='Rumbula')|(df['District']=='Suzi')),'B-District-riga-CITS')]=1
df['B-District-riga-Ilģuciems']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='dzeguzhkalns')|(df['District']=='ilguciems')|(df['District']=='Ilguciems')|(df['District']=='Dzirciems')|(df['District']=='Spilve')),'B-District-riga-Ilģuciems')]=1
df['B-District-riga-Imanta']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='imanta')|(df['District']=='kleisti')|(df['District']=='zasulauks')|(df['District']=='Imanta')|(df['District']=='Zasulauks')),'B-District-riga-Imanta')]=1
df['B-District-riga-Jugla']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='bergi')|(df['District']=='breksi')|(df['District']=='bukulti')|(df['District']=='yugla')|(df['District']=='Bergi')|(df['District']=='Jugla')|(df['District']=='Breksi')|(df['District']=='Bukulti')),'B-District-riga-Jugla')]=1
df['B-District-riga-Ķengarags']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='kengarags')|(df['District']=='Kengarags')|(df['District']=='Krasta')),'B-District-riga-Ķengarags')]=1
df['B-District-riga-Maskavas Forštate']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='krasta-st-area')|(df['District']=='maskavas-priekshpilseta')|(df['District']=='Maskavas')),'B-District-riga-Maskavas Forštate')]=1
df['B-District-riga-Mežaparkse']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='aplokciems')|(df['District']=='jaunciems')|(df['District']=='mezhapark')|(df['District']=='Aplokciems')|(df['District']=='Jaunciems')|(df['District']=='Mezaparks')),'B-District-riga-Mežaparkse')]=1
df['B-District-riga-Mežciems']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='dreilini')|(df['District']=='mezhciems')|(df['District']=='Dreilini')|(df['District']=='Mezciems')),'B-District-riga-Mežciems')]=1
df['B-District-riga-Pļavnieki']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='darzciems')|(df['District']=='plyavnieki')|(df['District']=='shkirotava')|(df['District']=='Darzciems')|(df['District']=='Plavnieki')|(df['District']=='Skirotava')),'B-District-riga-Pļavnieki')]=1
df['B-District-riga-Purvciems']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='purvciems')|(df['District']=='Purvciems')),'B-District-riga-Purvciems')]=1
df['B-District-riga-Sarkandaugava']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='kundzinsala')|(df['District']=='sarkandaugava')|(df['District']=='Sarkandaugava')|(df['District']=='Brasa')|(df['District']=='petersala')|(df['District']=='Skanste')),'B-District-riga-Sarkandaugava')]=1
df['B-District-riga-Teika']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='chiekurkalns')|(df['District']=='teika')|(df['District']=='vef')|(df['District']=='Teika')|(df['District']=='Ciekurkalns')),'B-District-riga-Teika')]=1
df['B-District-riga-Vecmīlgrāvis']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='mangali')|(df['District']=='mangalsala')|(df['District']=='vecaki')|(df['District']=='vecdaugava')|(df['District']=='vecmilgravis')|(df['District']=='Mangalu')|(df['District']=='Vecaki')|(df['District']=='Vecdaugava')|(df['District']=='Vecmilgravis')|(df['District']=='Jaunmilgravis')|(df['District']=='Milgravis')|(df['District']=='Trisciems')),'B-District-riga-Vecmīlgrāvis')]=1
df['B-District-riga-Ziepniekkalns']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='bierini')|(df['District']=='katlakalns')|(df['District']=='ziepniekkalns')|(df['District']=='Bierini')|(df['District']=='Katlakalns')|(df['District']=='Ziepniekkalns')|(df['District']=='Atgazene')|(df['District']=='Bisumuiza')),'B-District-riga-Ziepniekkalns')]=1
df['B-District-riga-Zolitūde']=0
df.loc[((df['Region adj']=='riga')&((df['District']=='shampeteris-pleskodale')|(df['District']=='zolitude')|(df['District']=='Zolitude')|(df['District']=='Pleskodale')),'B-District-riga-Zolitūde')]=1

# Approprate format for variables
df['ID']=df['ID'].astype(str)
cols = ['Prices','Land','Premise','Number of rooms','Floor','Max floor','Number of floors','Year','Month','Day','Quarter','B-ESTATE_TYPE-farms-estates','B-ESTATE_TYPE-flats','B-ESTATE_TYPE-homes-summer-residences','B-ESTATE_TYPE-offices','B-ESTATE_TYPE-plots-and-lands','B-TRANSACTION_TYPE-hand-over','B-TRANSACTION_TYPE-sell','B-HOUSE_TYPE-103.','B-HOUSE_TYPE-104.','B-HOUSE_TYPE-119.','B-HOUSE_TYPE-467.','B-HOUSE_TYPE-602.','B-HOUSE_TYPE-Čehu pr.','B-HOUSE_TYPE-Hrušč.','B-HOUSE_TYPE-Jaun.','B-HOUSE_TYPE-LT proj.','B-HOUSE_TYPE-M. ģim.','B-HOUSE_TYPE-P. kara','B-HOUSE_TYPE-Priv. m.','B-HOUSE_TYPE-Renov.','B-HOUSE_TYPE-Specpr.','B-HOUSE_TYPE-Staļina','B-District-riga-Āgenskalns','B-District-riga-Boldērāja','B-District-riga-Centrs','B-District-riga-CITS','B-District-riga-Ilģuciems','B-District-riga-Imanta','B-District-riga-Jugla','B-District-riga-Ķengarags','B-District-riga-Maskavas Forštate','B-District-riga-Mežaparkse','B-District-riga-Mežciems','B-District-riga-Pļavnieki','B-District-riga-Purvciems','B-District-riga-Sarkandaugava','B-District-riga-Teika','B-District-riga-Vecmīlgrāvis','B-District-riga-Ziepniekkalns','B-District-riga-Zolitūde']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
del cols

df['Duration date']=df['Duration date'].apply(pd.to_numeric)
df.loc[(df['Duration date']>150,'Duration')]='>150'
df.loc[((df['Duration date']<=150)&(df['Duration date']>50),'Duration')]='51-150'
df.loc[((df['Duration date']<=50)&(df['Duration date']>7),'Duration')]='8-50'
df.loc[((df['Duration date']<=7)&(df['Duration date']>=2),'Duration')]='2-7'
df.loc[((df['Duration date']<=1)&(df['Duration date']>0),'Duration')]='1'
df.loc[((df['Duration date']==0),'Duration')]='0'

df.loc[(df['Duration']=='>150','B-Duration->150')]=1
df.loc[(df['Duration']!='>150','B-Duration->150')]=0
df.loc[(df['Duration']=='51-150','B-Duration-51-150')]=1
df.loc[(df['Duration']!='51-150','B-Duration-51-150')]=0
df.loc[(df['Duration']=='8-50','B-Duration-8-50')]=1
df.loc[(df['Duration']!='8-50','B-Duration-8-50')]=0
df.loc[(df['Duration']=='2-7','B-Duration-2-7')]=1
df.loc[(df['Duration']!='2-7','B-Duration-2-7')]=0
df.loc[(df['Duration']=='1','B-Duration-1')]=1
df.loc[(df['Duration']!='1','B-Duration-1')]=0
df.loc[(df['Duration']=='0','B-Duration-0')]=1
df.loc[(df['Duration']!='0','B-Duration-0')]=0

# Export
conn = sqlite3.connect('database.db')
df.to_sql('df', conn, if_exists='replace', index = False)

In [ ]:
import sklearn
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import statsmodels.api as sm

import pandas as pd
import sqlite3
con = sqlite3.connect("database.db")
dff = pd.read_sql_query("SELECT * from df", con)
print(dff.head()) #for verification
con.close()

def make_regression(dataframe, estate_type, transaction_type, region_type,rent_freq):   
    print("xx"+ estate_type)
    #estate_type = farms, residences, plots, offices, flats, flats2 (with house types)
    #transaction_type = sell, hand_over
    #region_type = riga, riga-region, cities, other
        if estate_type=='farms':
        dataframe=dataframe[dataframe['ESTATE_TYPE']== 'farms-estates']
    elif estate_type=='residences':
        dataframe=dataframe[dataframe['ESTATE_TYPE']== 'homes-summer-residences']
    elif estate_type=='plots':
        dataframe=dataframe[dataframe['ESTATE_TYPE']== 'plots-and-lands']
    elif estate_type=='offices':
        dataframe=dataframe[dataframe['ESTATE_TYPE']== 'offices']
    elif estate_type=='flats' or estate_type=='flats2':
        dataframe=dataframe[dataframe['ESTATE_TYPE']== 'flats']
    else:
        print('Check a name of estate_type')
    
    if transaction_type=='sell' or transaction_type=='hand_over':
        dataframe=dataframe[dataframe['TRANSACTION_TYPE']== transaction_type]
    else:
        print('Check a name of transaction_type')

    if region_type=='riga' or region_type=='riga-region' or region_type=='cities' or region_type=='other':
        dataframe=dataframe[dataframe['Region adj']== region_type]
    else:
        print('Check a name of region_type')    

    if rent_freq=='full':
        dataframe=dataframe[dataframe['Currency format']=='€']
    elif rent_freq=='per month':
        dataframe=dataframe[dataframe['Currency format']=='€/mēn.']
    elif rent_freq=='per week':
        dataframe=dataframe[dataframe['Currency format']=='€/ned.']
    elif rent_freq=='per day':
        dataframe=dataframe[dataframe['Currency format']=='€/dienā']
    else:
        print('Check a name of type_price')   


    if dataframe.empty:
        pass
    elif len(dataframe.index) < 15:
        pass
    else:
        results = do_regression(dataframe, estate_type, transaction_type, region_type,rent_freq)
        return results

#define function
#function will use entered dataframe as a source and make necessary adjustments depending on transaction type (sale, rent)
# and depending on real estate type

def do_regression(dataframe, estate_type, transaction_type, region_type, rent_freq):   
    print(estate_type + transaction_type +region_type+rent_freq)
    if estate_type == "farms" or estate_type == "plots":
        dataframe = dataframe[np.abs(dataframe.Priceland-dataframe.Priceland.mean()) <= (3*dataframe.Priceland.std())]
        dataframe=dataframe[~(np.abs(dataframe.Priceland-dataframe.Priceland.mean()) > (3*dataframe.Priceland.std()))]
        
    elif estate_type == "offices" or estate_type == "flats" or estate_type == "flats2" or estate_type=="residences":
        dataframe=dataframe[np.abs(dataframe.Pricepremise-dataframe.Pricepremise.mean()) <= (3*dataframe.Pricepremise.std())]
        dataframe=dataframe[~(np.abs(dataframe.Pricepremise-dataframe.Pricepremise.mean()) > (3*dataframe.Pricepremise.std()))]
    
    correlation=dataframe.corr()
    correlation=correlation.round(4)

    #Lists
    list_riga=['B-District-riga-Āgenskalns','B-District-riga-Boldērāja','B-District-riga-Centrs','B-District-riga-Ilģuciems','B-District-riga-Imanta','B-District-riga-Jugla','B-District-riga-Ķengarags','B-District-riga-Maskavas Forštate','B-District-riga-Mežaparkse','B-District-riga-Mežciems','B-District-riga-Pļavnieki','B-District-riga-Purvciems','B-District-riga-Sarkandaugava','B-District-riga-Teika','B-District-riga-Vecmīlgrāvis','B-District-riga-Ziepniekkalns','B-District-riga-Zolitūde']
    list_house_types=['B-HOUSE_TYPE-103.','B-HOUSE_TYPE-104.','B-HOUSE_TYPE-119.','B-HOUSE_TYPE-467.','B-HOUSE_TYPE-602.','B-HOUSE_TYPE-Čehu pr.','B-HOUSE_TYPE-Hrušč.','B-HOUSE_TYPE-Jaun.','B-HOUSE_TYPE-LT proj.','B-HOUSE_TYPE-M. ģim.','B-HOUSE_TYPE-P. kara','B-HOUSE_TYPE-Priv. m.','B-HOUSE_TYPE-Renov.','B-HOUSE_TYPE-Specpr.']
    list_duration=['B-Duration-0','B-Duration-1','B-Duration-2-7','B-Duration-8-50','B-Duration-51-150']

    list_farms_0=['Land','Premise','Priceland','Pricepremise','Number of floors','Quarter']
    list_residences_0=['Land','Premise','Priceland','Pricepremise','Number of floors','Quarter']
    list_plots_0=['Land','Priceland','Quarter']
    list_offices_0=['Premise','Pricepremise','Floor','Max floor','Quarter']
    list_flats_0=['Premise','Pricepremise','Number of rooms','Quarter']

    if estate_type == 'farms': 
        list1=list_farms_0
    elif estate_type == 'residences': 
        list1=list_residences_0
    elif  estate_type == 'offices': 
        list1=list_offices_0
    elif  estate_type == 'plots': 
        list1=list_plots_0
    elif  estate_type == 'flats': 
        list1=list_flats_0
    elif  estate_type == 'flats2': 
        list1=list_flats_0+list_house_types

    list1=list1+list_duration

    if region_type=='riga':
        list1=list1+list_riga
    
    if estate_type == 'farms' or estate_type == 'residences' or estate_type == 'offices' or estate_type == 'flats' or estate_type == 'flats2':
        proportion_no_premise_data=(dataframe['Premise'].isna().sum())/dataframe['Premise'].count()
        if proportion_no_premise_data<0.01:
            print('yes', proportion_no_premise_data)
            dataframe=dataframe.dropna(subset=['Premise'])
        else:
            print('no')
            print("yy" + str(proportion_no_premise_data)+ str(dataframe['Premise'].mean()))
            dataframe['Premise']=dataframe['Premise'].fillna(dataframe['Premise'].mean())
            dataframe['Pricepremise']=dataframe['Prices']/dataframe['Premise']

    if estate_type == 'farms' or estate_type == 'residences' or estate_type == 'plots':
        proportion_no_land_data=(dataframe['Land'].isna().sum())/dataframe['Land'].count()
        if proportion_no_land_data<0.01:
            print('yes', proportion_no_land_data)
            dataframe=dataframe.dropna(subset=['Land'])
        else:
            print('no')
            print("yy" + str(proportion_no_land_data)+ str(dataframe['Land'].mean()))
            dataframe['Land']=dataframe['Land'].fillna(dataframe['Land'].mean())
            dataframe['Priceland']=dataframe['Prices']/dataframe['Land']
    
    if estate_type == 'farms' or estate_type == 'residences':
        proportion_no_floors_data=(dataframe['Number of floors'].isna().sum())/dataframe['Number of floors'].count()
        if proportion_no_floors_data<0.01:
            print('yes', proportion_no_floors_data)
            dataframe=dataframe.dropna(subset=['Number of floors'])
        else:
            print('no')
            print("yy" + str(proportion_no_floors_data)+ str(dataframe['Number of floors'].mean()))
            dataframe['Number of floors']=dataframe['Number of floors'].fillna(dataframe['Number of floors'].mean())
    
    elif estate_type == 'flats' or estate_type == 'flats2':
        proportion_no_rooms_data=(dataframe['Number of rooms'].isna().sum())/dataframe['Number of rooms'].count()
        if proportion_no_rooms_data<0.01:
            print('yes', proportion_no_rooms_data)
            dataframe=dataframe.dropna(subset=['Number of rooms'])
        else:
            print('no')
            print("yy" + str(proportion_no_rooms_data)+ str(dataframe['Number of rooms'].mean()))
            dataframe['Number of rooms']=dataframe['Number of rooms'].fillna(dataframe['Number of rooms'].mean())

    elif estate_type == 'offices':
        proportion_no_max_floor_data=(dataframe['Max floor'].isna().sum())/dataframe['Max floor'].count()
        if proportion_no_max_floor_data<0.01:
            print('yes', proportion_no_max_floor_data)
            dataframe=dataframe.dropna(subset=['Max floor'])
        else:
            print('no')
            print("yy" + str(proportion_no_max_floor_data)+ str(dataframe['Max floor'].mean()))
            dataframe['Max floor']=dataframe['Max floor'].fillna(dataframe['Max floor'].mean())

        proportion_no_floor_data=(dataframe['Floor'].isna().sum())/dataframe['Floor'].count()
        if proportion_no_floor_data<0.01:
            print('yes', proportion_no_floor_data)
            dataframe=dataframe.dropna(subset=['Floor'])
        else:
            print('no')
            print("yy" + str(proportion_no_floor_data)+ str(dataframe['Floor'].mean()))
            dataframe['Floor']=dataframe['Floor'].fillna(dataframe['Floor'].mean())
    
    #Summary
    y = dataframe['Prices']
    x = dataframe[list1]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    summary_as_html = model.summary().tables[1].as_html()
    summary=pd.read_html(summary_as_html, header=0, index_col=0)[0]
    summary=summary.rename(columns={'coef':'Value of determinant','std err':'Standard deviation','[0.025':'95% Confidence','0.975]':'interval'})

    if  transaction_type=='hand_over' and ((estate_type == "farms" and rent_freq=='per month' and (region_type == "riga-region" or region_type == "cities")) or (estate_type == "offices" and ((rent_freq=='per week' and region_type == "riga") or (rent_freq=='per day' and (region_type == "riga-region" or region_type == "cities" or region_type == "other"))))or(estate_type == "plots" and rent_freq=='per day' and (region_type == "riga-region" or region_type=='other'))):
        pass
    else:
        list1.insert(0,'Intercept')
        summary.insert(loc=0,column='Name of determinant',value=list1)

    #Creating new column "Prices adjusted"
    y_predicted=model.predict(x)
    dataframe['Prices adjusted']=y_predicted
    dataframe['Prices adjusted']=dataframe['Prices adjusted'].round(2)

    #regression scores
    rscore=model.rsquared
    score=pd.DataFrame({'Regression score': [rscore]})
    score['Regression score']=score['Regression score'].round(4)

    return dataframe, correlation, summary, score

#make_regression(dff, str('flats'), str('hand_over'),str('other'),str('per day'))

#create table index that is dataframe
tmin=dff['Year'].astype(int).min()
tmax=1+dff['Year'].astype(int).max()
number_years=tmax-tmin
years = [2020]*960+ [2021]*960+[2022]*960
month=[1,2,3,4,5,6,7,8,9,10,11,12]*80*number_years
transaction_type=(['sell']*240+['hand_over']*720)*number_years
estate_type=(['farms']*48+['residences']*48+['offices']*48+['plots']*48+['flats']*48+['farms']*3*48+['residences']*3*48+['offices']*3*48+['plots']*3*48+['flats']*3*48)*number_years
region=((['riga']*12+['riga-region']*12+['cities']*12+['other']*12)*5+(['riga']*3*12+['riga-region']*3*12+['cities']*3*12+['other']*3*12)*5)*number_years
rent_freq=(['full']*240+(['per month']*12+['per week']*12+['per day']*12)*4*5)*number_years

table = {'Year' : years,'Month':month,'Transaction type':transaction_type,'Estate type':estate_type,'Region': region,'Type of price':rent_freq}
table_index=pd.DataFrame(table)
xmax=table_index.shape[0]

#Prices, number of rows, Index 1
def update_table_index_1(table_index, dataframe, transaction_type, estate_type, region_type, rent_freq = "full"):
    for t in range (2020,2023):
        for i in range (1,13):
            if estate_type != "plots":
                mean_pricepremise = dataframe.loc[(dataframe['Year']==t)&(dataframe['Month']==i),'Pricepremise'].mean()
            else:
                mean_pricepremise = 0
            if estate_type != "offices" and (estate_type != "flats" and estate_type != "flats2"):
                mean_priceland = dataframe.loc[(dataframe['Year']==t)&(dataframe['Month']==i),'Priceland'].mean()
            else:
                mean_priceland = 0
            mean_prices = dataframe.loc[(dataframe['Year']==t)&(dataframe['Month']==i),'Prices'].mean()
            row_count = dataframe.loc[(dataframe['Prices']!='')&(dataframe['Year']==t)&(dataframe['Month']==i),'Prices'].count()
            entry_location = table_index.index[(table_index['Year']==t)&(table_index['Month']==i)&(table_index['Transaction type']==transaction_type)&(table_index['Estate type']==estate_type)&(table_index['Region']==region_type)&(table_index['Type of price']==rent_freq)]
            print("index" + str(entry_location))
            table_index.at[entry_location,1] =mean_pricepremise
            table_index.at[entry_location,2] =mean_priceland
            table_index.at[entry_location,3] =mean_prices

            if (t == 2020 and i == 1) or (estate_type == "flats2"):
                table_index.at[entry_location,4] = 0
                table_index.at[entry_location,5] = 0
                table_index.at[entry_location,6] = 0
                pass
            else:
                if i == 1:
                    previous_i = 12
                    previous_t = t - 1
                else:
                    previous_i = i - 1
                    previous_t = t 

                entry_location_month_before = table_index.index[(table_index['Year']==previous_t)&(table_index['Month']==previous_i)&(table_index['Transaction type']==transaction_type)&(table_index['Estate type']==estate_type)&(table_index['Region']==region_type)&(table_index['Type of price']==rent_freq)]
                if (table_index.iloc[entry_location,6].item() != 0) and (table_index.iloc[entry_location_month_before,6].item() != 0):
                    change_pricepremise = (table_index.iloc[entry_location,6].item()/table_index.iloc[entry_location_month_before,6].item())-1
                else:
                    change_pricepremise = ''
                if (table_index.iloc[entry_location,7].item() != 0) and (table_index.iloc[entry_location_month_before,7].item() != 0):
                    change_priceland = (table_index.iloc[entry_location,7].item()/table_index.iloc[entry_location_month_before,7].item())-1
                else:
                    change_priceland = ''
                if (table_index.iloc[entry_location,8].item() != 0) and (table_index.iloc[entry_location_month_before,8].item() != 0):
                    change_prices = (table_index.iloc[entry_location,8].item()/table_index.iloc[entry_location_month_before,8].item())-1
                else:
                    change_prices = ''

                table_index.at[entry_location,4] = change_pricepremise
                table_index.at[entry_location,5] = change_priceland
                table_index.at[entry_location,6] = change_prices
            table_index.at[entry_location,7] = row_count
            table_index[7]=table_index[7].replace(np.nan, 0)

    return table_index

# lists thorugh which to iterate
list_transactions = ["sell", "hand_over"]
list_estate_types = ["farms", "residences", "offices", "plots", "flats"]
list_regions = ["riga", "riga-region", "cities", "other"]
list_rent_freq = ["per month", "per week", "per day"]

for transaction_type in range(len(list_transactions)):
    dataframe=dff
    for estate_type in range(len(list_estate_types)):
        for region_type in range(len(list_regions)):            
            if list_transactions[transaction_type] == "hand_over":
                for rent_freq in range(len(list_rent_freq)):
                    results = make_regression(dataframe[(dataframe['Region adj']==list_regions[region_type])], list_estate_types[estate_type], list_transactions[transaction_type], list_regions[region_type],list_rent_freq[rent_freq])
                    if results != None:
                        result_df = results[0]
                        table_index = update_table_index_1(table_index, result_df, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type], list_rent_freq[rent_freq])
            else:
                print(list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type])
                results = make_regression(dataframe[dataframe['Region adj']==list_regions[region_type]], list_estate_types[estate_type], list_transactions[transaction_type], list_regions[region_type],'full')
                if results != None:
                    result_df = results[0]                    
                    table_index = update_table_index_1(table_index, result_df, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type],'full')

#Share regions
def update_table_index_share_regions(table_index):    
    for t in range (2020,2023):
        for i in range (1,13):
            for transaction in list_transactions:
                for estate in list_estate_types:            
                    if transaction == "sell":
                        sum_region = 0
                        for region in list_regions:
                            entry_location_temp = table_index.index[(table_index['Year']==t)&(table_index['Month']==i)&(table_index['Transaction type']==transaction)&(table_index['Estate type']==estate)&(table_index['Region']==region)&(table_index['Type of price']=="full")]
                            print(t, i, transaction, estate, region)
                            if table_index.iloc[entry_location_temp,12].empty:
                                pass
                            else:
                                sum_region = sum_region + table_index.iloc[entry_location_temp,12].item()
                        if sum_region != 0:
                            for region in list_regions:
                                entry_location_temp = table_index.index[(table_index['Year']==t)&(table_index['Month']==i)&(table_index['Transaction type']==transaction)&(table_index['Estate type']==estate)&(table_index['Region']==region)&(table_index['Type of price']=="full")]
                                if table_index.iloc[entry_location_temp,12].empty:
                                    table_index.at[entry_location_temp,8] = 0
                                else:
                                    table_index.at[entry_location_temp,8] = table_index.iloc[entry_location_temp,12].item() / sum_region
                        else:
                            pass
                    else:
                        for freq in list_rent_freq:
                            sum_region = 0
                            for region in list_regions:
                                entry_location_temp = table_index.index[(table_index['Year']==t)&(table_index['Month']==i)&(table_index['Transaction type']==transaction)&(table_index['Estate type']==estate)&(table_index['Region']==region)&(table_index['Type of price']==freq)]
                                if table_index.iloc[entry_location_temp,12].empty:
                                    pass
                                else:
                                    sum_region = sum_region + table_index.iloc[entry_location_temp,12].item()
                                    if sum_region != 0:
                                        for region in list_regions:
                                            entry_location_temp = table_index.index[(table_index['Year']==t)&(table_index['Month']==i)&(table_index['Transaction type']==transaction)&(table_index['Estate type']==estate)&(table_index['Region']==region)&(table_index['Type of price']==freq)]
                                            table_index.at[entry_location_temp,8] = table_index.iloc[entry_location_temp,12].item() / sum_region
                                        else:
                                            pass                
    for i in range(4,8):
        table_index[i]=table_index[i].replace(np.nan, 0)
        table_index[i]=table_index[i].replace('', 0)
    return table_index

#Index 2
def update_table_index_2(table_index):
    for t in range (2020,2023):
        for i in range (1,13):
            for transaction in list_transactions:
                for estate in list_estate_types:            
                    for price_type in range(0,3):
                        if transaction == "sell":
                            index2 = 0
                            for region in list_regions:
                                entry_location_temp = table_index.index[(table_index['Year']==t)&(table_index['Month']==i)&(table_index['Transaction type']==transaction)&(table_index['Estate type']==estate)&(table_index['Region']==region)&(table_index['Type of price']=="full")]
                                if table_index.iloc[entry_location_temp,13].empty or table_index.iloc[entry_location_temp,9+price_type].empty:
                                    add_index2 = 0
                                else:
                                    add_index2 = table_index.iloc[entry_location_temp,13].item() * table_index.iloc[entry_location_temp,9+price_type].item() 
                                index2 = index2 + add_index2
                            for region in list_regions:
                                entry_location_temp = table_index.index[(table_index['Year']==t)&(table_index['Month']==i)&(table_index['Transaction type']==transaction)&(table_index['Estate type']==estate)&(table_index['Region']==region)&(table_index['Type of price']=="full")]                  
                                table_index.at[entry_location_temp,9+price_type] = index2
                        else:
                            for freq in list_rent_freq:
                                index2 = 0
                                for region in list_regions:
                                    entry_location_temp = table_index.index[(table_index['Year']==t)&(table_index['Month']==i)&(table_index['Transaction type']==transaction)&(table_index['Estate type']==estate)&(table_index['Region']==region)&(table_index['Type of price']==freq)]
                                    if table_index.iloc[entry_location_temp,13].empty or table_index.iloc[entry_location_temp,9+price_type].empty:
                                        add_index2 = 0
                                    else:
                                        add_index2 = table_index.iloc[entry_location_temp,13].item() * table_index.iloc[entry_location_temp,9+price_type].item() 
                                    index2 = index2 + add_index2
                                for region in list_regions:
                                    entry_location_temp = table_index.index[(table_index['Year']==t)&(table_index['Month']==i)&(table_index['Transaction type']==transaction)&(table_index['Estate type']==estate)&(table_index['Region']==region)&(table_index['Type of price']==freq)]                  
                                    table_index.at[entry_location_temp,9+price_type] = index2

                
    return table_index
table_index = update_table_index_share_regions(table_index)
table_index = update_table_index_2(table_index)

#Share for estate types
#for sell
for x in range(0,xmax,960):
    for i in range(0,12):
        sum_sell_farms=table_index.at[0+i+x,7]+table_index.at[0+12+i+x,7]+table_index.at[0+24+i+x,7]+table_index.at[0+36+i+x,7]
        sum_sell_residences=table_index.at[48+i+x,7]+table_index.at[48+12+i+x,7]+table_index.at[48+24+i+x,7]+table_index.at[48+36+i+x,7]
        sum_sell_offices=table_index.at[48*2+i+x,7]+table_index.at[48*2+12+i+x,7]+table_index.at[48*2+24+i+x,7]+table_index.at[48*2+36+i+x,7]
        sum_sell_plots_lands=table_index.at[48*3+i+x,7]+table_index.at[48*3+12+i+x,7]+table_index.at[48*3+24+i+x,7]+table_index.at[48*3+36+i+x,7]
        sum_sell_flats=table_index.at[48*4+i+x,7]+table_index.at[48*4+12+i+x,7]+table_index.at[48*4+24+i+x,7]+table_index.at[48*4+36+i+x,7]
        sum=sum_sell_farms+sum_sell_residences+sum_sell_offices+sum_sell_plots_lands+sum_sell_flats
        sum_premise=sum_sell_farms+sum_sell_residences+sum_sell_offices+sum_sell_flats
        sum_land=sum_sell_farms+sum_sell_residences+sum_sell_plots_lands
        
        for p in range(0,37,12):
            table_index.at[p+i+x,12]=sum_sell_farms/sum_premise
            table_index.at[p+i+x,13]=sum_sell_farms/sum_land
            table_index.at[p+i+x,14]=sum_sell_farms/sum
        for p in range(48,85,12):
            table_index.at[p+i+x,12]=sum_sell_residences/sum_premise
            table_index.at[p+i+x,13]=sum_sell_residences/sum_land
            table_index.at[p+i+x,14]=sum_sell_residences/sum
        for p in range(96,133,12):
            table_index.at[p+i+x,12]=sum_sell_offices/sum_premise
            table_index.at[p+i+x,14]=sum_sell_offices/sum
        for p in range(144,181,12):
            table_index.at[p+i+x,13]=sum_sell_plots_lands/sum_land
            table_index.at[p+i+x,14]=sum_sell_plots_lands/sum
        for p in range(192,229,12):
            table_index.at[p+i+x,12]=sum_sell_flats/sum_premise
            table_index.at[p+i+x,14]=sum_sell_flats/sum
        
#for hand over
for x in range(0,xmax,960):
    for i in range(0,12):
        for v in range(0,3):
            sum_hand_over_farms=table_index.at[240+i+x+12*v,7]+table_index.at[240+36+i+x+12*v,7]+table_index.at[240+36*2+i+x+12*v,7]+table_index.at[240+36*3+i+x+12*v,7]
            sum_hand_over_residences=table_index.at[240+144*1+i+x+12*v,7]+table_index.at[240+144*1+36+i+x+12*v,7]+table_index.at[240+144*1+36*2+i+x+12*v,7]+table_index.at[240+144*1+36*3+i+x+12*v,7]
            sum_hand_over_offices=table_index.at[240+144*2+i+x+12*v,7]+table_index.at[240+144*2+36+i+x+12*v,7]+table_index.at[240+144*2+36*2+i+x+12*v,7]+table_index.at[240+144*2+36*3+i+x+12*v,7]
            sum_hand_over_plots_lands=table_index.at[240+144*3+i+x+12*v,7]+table_index.at[240+144*3+36+i+x+12*v,7]+table_index.at[240+144*3+36*2+i+x+12*v,7]+table_index.at[240+144*3+36*3+i+x+12*v,7]
            sum_hand_over_flats=table_index.at[240+144*4+i+x+12*v,7]+table_index.at[240+144*4+36+i+x+12*v,7]+table_index.at[240+144*4+36*2+i+x+12*v,7]+table_index.at[240+144*4+36*3+i+x+12*v,7]
            sum=sum_hand_over_farms+sum_hand_over_residences+sum_hand_over_offices+sum_hand_over_plots_lands+sum_hand_over_flats
            sum_premise=sum_hand_over_farms+sum_hand_over_residences+sum_hand_over_offices+sum_hand_over_flats
            sum_land=sum_hand_over_farms+sum_hand_over_residences+sum_hand_over_plots_lands
            
            for p in range(0,109,36):
                table_index.at[240+i+x+12*v+p,12]=sum_hand_over_farms/sum_premise
                table_index.at[240+i+x+12*v+p,13]=sum_hand_over_farms/sum_land
                table_index.at[240+i+x+12*v+p,14]=sum_hand_over_farms/sum
            for p in range(144,253,36):
                table_index.at[240+i+x+12*v+p,12]=sum_hand_over_residences/sum_premise
                table_index.at[240+i+x+12*v+p,13]=sum_hand_over_residences/sum_land
                table_index.at[240+i+x+12*v+p,14]=sum_hand_over_residences/sum
            for p in range(288,397,36):
                table_index.at[240+i+x+12*v+p,12]=sum_hand_over_offices/sum_premise
                table_index.at[240+i+x+12*v+p,14]=sum_hand_over_offices/sum
            for p in range(432,541,36):
                table_index.at[240+i+x+12*v+p,13]=sum_hand_over_plots_lands/sum_land
                table_index.at[240+i+x+12*v+p,14]=sum_hand_over_plots_lands/sum
            for p in range(576,685,36):
                table_index.at[240+i+x+12*v+p,12]=sum_hand_over_flats/sum_premise
                table_index.at[240+i+x+12*v+p,14]=sum_hand_over_flats/sum
                
print(table_index)

# Index 3
# according to each type of price (price per premise, price per land, price)
for i in range(8,15):
    table_index[i]=table_index[i].replace(np.nan, 0)
    table_index[i]=table_index[i].replace('', 0)

#for sell
for x in range(0,xmax,960):
    for i in range(0,12):
        for w in range(0,3):
            index=table_index.at[0+i+x,9+w]*table_index.at[0+i+x,12+w]+table_index.at[0+48*1+i+x,9+w]*table_index.at[0+48*1+i+x,12+w]+table_index.at[0+48*2+i+x,9+w]*table_index.at[0+48*2+i+x,12+w]+table_index.at[0+48*3+i+x,9+w]*table_index.at[0+48*3+i+x,12+w]+table_index.at[0+48*4+i+x,9+w]*table_index.at[0+48*4+i+x,12+w]
            for p in range(0,229,12):
                table_index.at[0+i+x+p,15+w]=index
#for hand over
for x in range(0,xmax,960):
    for i in range(0,12):
        for w in range(0,3):
            for v in range(0,3):
                index=table_index.at[240+i+x+12*v,9+w]*table_index.at[240+i+x+12*v,12+w]+table_index.at[240+144*1+i+x+12*v,9+w]*table_index.at[240+144*1+i+x+12*v,12+w]+table_index.at[240+144*2+i+x+12*v,9+w]*table_index.at[240+144*2+i+x+12*v,12+w]+table_index.at[240+144*3+i+x+12*v,9+w]*table_index.at[240+144*3+i+x+12*v,12+w]+table_index.at[240+144*4+i+x+12*v,9+w]*table_index.at[240+144*4+i+x+12*v,12+w]
                for p in range(240,960,36):
                    table_index.at[0+i+x+12*v+p,15+w]=index

# according to type of price that suits the most for an appropriate estate type (price per premise - flats, residences,offices; price per land - farms, plots and lands)
#for sell
for x in range(0,xmax,960):
    for i in range(0,12):
            index=table_index.at[0+i+x,10]*table_index.at[0+i+x,14]+table_index.at[0+48*1+i+x,9]*table_index.at[0+48*1+i+x,14]+table_index.at[0+48*2+i+x,9]*table_index.at[0+48*2+i+x,14]+table_index.at[0+48*3+i+x,10]*table_index.at[0+48*3+i+x,14]+table_index.at[0+48*4+i+x,9]*table_index.at[0+48*4+i+x,14]
            for p in range(0,229,12):
                table_index.at[0+i+x+p,18]=index

#for hand over
for x in range(0,xmax,960):
    for i in range(0,12):
            for v in range(0,3):
                index=table_index.at[240+i+x+12*v,10]*table_index.at[240+i+x+12*v,14]+table_index.at[240+144*1+i+x+12*v,9]*table_index.at[240+144*1+i+x+12*v,14]+table_index.at[240+144*2+i+x+12*v,9]*table_index.at[240+144*2+i+x+12*v,14]+table_index.at[240+144*3+i+x+12*v,10]*table_index.at[240+144*3+i+x+12*v,14]+table_index.at[240+144*4+i+x+12*v,9]*table_index.at[240+144*4+i+x+12*v,14]
                for p in range(240,960,36):
                    table_index.at[0+i+x+12*v+p,18]=index
                    
print(table_index)

table_index=table_index.rename(columns={1:'Price per premise',2:'Price per land',3:'Price',
                            4:'Index price per premise',5:'Index price per land',6:'Index price',
                            7:'Number of rows',8:'Share for region types',
                            9:'Index price estate type per premise',10:'Index price estate type per land',11:'Index price estate type',
                            
                            12:'Share for estate types (premise)',13:'Share for estate types (land)',14:'Share for estate types',

                            15:'Index price transaction type per premise',16:'Index price transaction type per land',17:'Index price transaction type',
                            18:'Index price transaction type adjusted'})

# %%
conn = sqlite3.connect('database.db')
table_index.to_sql('table_index', conn, if_exists='replace', index = False)
#table_index.to_csv('index_js.csv')

# %%
dataframe_table = pd.DataFrame()
correlation_table = pd.DataFrame()
summary_table = pd.DataFrame()
score_table = pd.DataFrame()

# %%
def update_dataframe_table(dataframe_table, dataframe, transaction_type, estate_type, region_type, rent_freq='full'):
    dataframe['Index']=str(transaction_type)+'__'+str(estate_type)+'__'+str(region_type)+'__'+str(rent_freq)
    dataframe_table=dataframe_table.append(dataframe,ignore_index=True)
    return dataframe_table

# %%
def update_correlation_table(correlation_table, correlation, transaction_type, estate_type, region_type, rent_freq='full'):   
    correlation0=correlation.iloc[[0]]
    correlation0['Index']=str(transaction_type)+'__'+str(estate_type)+'__'+str(region_type)+'__'+str(rent_freq)
    correlation_table=correlation_table.append(correlation0,ignore_index=True)
    return correlation_table

# %%
def update_summary_table(summary_table, summary, transaction_type, estate_type, region_type, rent_freq='full'):
    summary['Index']=str(transaction_type)+'__'+str(estate_type)+'__'+str(region_type)+'__'+str(rent_freq)
    summary_table=summary_table.append(summary,ignore_index=True)
    return summary_table

# %%
def update_score_table(score_table, score, transaction_type, estate_type, region_type, rent_freq='full'):
    score['Index']=str(transaction_type)+'__'+str(estate_type)+'__'+str(region_type)+'__'+str(rent_freq)
    score_table=score_table.append(score,ignore_index=True)
    return score_table

In [ ]:
# Regressions
list_transactions = ["sell", "hand_over"]
list_estate_types = ["farms", "residences", "offices", "plots", "flats", "flats2"]
list_regions = ["riga", "riga-region", "cities", "other"]
list_column_data =[] #this maybe should be modified more clearly
list_rent_freq = ["per month", "per week", "per day"]

for transaction_type in range(len(list_transactions)):
    dataframe=dff
    
    for estate_type in range(len(list_estate_types)):

        for region_type in range(len(list_regions)):            
            if list_transactions[transaction_type] == "hand_over":
                for rent_freq in range(len(list_rent_freq)):
                    results = make_regression(dataframe[(dataframe['Region adj']==list_regions[region_type])], list_estate_types[estate_type], list_transactions[transaction_type], list_regions[region_type],list_rent_freq[rent_freq])
                    if results != None:
                        result_dataframe = results[0]
                        result_correlation = results[1]
                        result_summary = results[2]
                        result_score = results[3]
                        if dataframe.empty:
                            pass
                        elif len(dataframe.index) < 10:
                            pass
                        else:
                            dataframe_table = update_dataframe_table(dataframe_table, result_dataframe, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type], list_rent_freq[rent_freq])
                            correlation_table = update_correlation_table(correlation_table, result_correlation, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type], list_rent_freq[rent_freq])
                            summary_table = update_summary_table(summary_table, result_summary, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type], list_rent_freq[rent_freq])
                            score_table = update_score_table(score_table, result_score, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type], list_rent_freq[rent_freq])
            else:
                print(list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type])
                if list_estate_types[estate_type] == "flats2":
                    results = make_regression(dataframe[(dataframe['AD_SOURCE']=='ss.com') & (dataframe['Region adj']==list_regions[region_type])], list_estate_types[estate_type], list_transactions[transaction_type], list_regions[region_type],'full')
                    if results != None:
                        result_dataframe = results[0]
                        result_correlation = results[1]
                        result_summary = results[2]
                        result_score = results[3]
                        if dataframe.empty:
                            pass
                        elif len(dataframe.index) < 10:
                            pass
                        else:
                            dataframe_table = update_dataframe_table(dataframe_table, result_dataframe, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type],'full')
                            correlation_table = update_correlation_table(correlation_table, result_correlation, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type],'full')
                            summary_table = update_summary_table(summary_table, result_summary, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type],'full')
                            score_table = update_score_table(score_table, result_score, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type],'full')
                else:
                    results = make_regression(dataframe[dataframe['Region adj']==list_regions[region_type]], list_estate_types[estate_type], list_transactions[transaction_type], list_regions[region_type],'full')
                    if results != None:
                        result_dataframe = results[0]
                        result_correlation = results[1]
                        result_summary = results[2]
                        result_score = results[3] 
                        if dataframe.empty:
                            pass
                        elif len(dataframe.index) < 10:
                            pass
                        else:
                            dataframe_table = update_dataframe_table(dataframe_table, result_dataframe, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type],'full')
                            correlation_table = update_correlation_table(correlation_table, result_correlation, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type],'full')
                            summary_table = update_summary_table(summary_table, result_summary, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type],'full')
                            score_table = update_score_table(score_table, result_score, list_transactions[transaction_type], list_estate_types[estate_type], list_regions[region_type],'full')

def update_Index(table):
    table[['Transaction type','Estate type','Region','Type of price']] =table['Index'].str.split('__',expand=True)
    table.loc[(table['Transaction type']=='hand_over','Transaction type')]='hand over'
    table.loc[(table['Estate type']=='flats2','Type of regression')]='with house types'
    table.loc[(table['Estate type']!='flats2','Type of regression')]='without house types'
    table.loc[(table['Estate type']=='flats2','Estate type')]='flats'
    table.loc[(table['Estate type']=='plots','Estate type')]='plots and lands'
    table.loc[(table['Region']=='riga-region','Region')]='riga region'
    return table

dataframe_table=update_Index(dataframe_table)
dataframe_table=dataframe_table.drop(['Region adj','TRANSACTION_TYPE','ESTATE_TYPE'],axis=1)

correlation_table=update_Index(correlation_table)
correlation_table.loc[(correlation_table['Type of regression']=='without house types',['B-HOUSE_TYPE-103.','B-HOUSE_TYPE-104.','B-HOUSE_TYPE-119.','B-HOUSE_TYPE-467.','B-HOUSE_TYPE-602.','B-HOUSE_TYPE-Čehu pr.','B-HOUSE_TYPE-Hrušč.','B-HOUSE_TYPE-Jaun.','B-HOUSE_TYPE-LT proj.','B-HOUSE_TYPE-M. ģim.','B-HOUSE_TYPE-P. kara','B-HOUSE_TYPE-Priv. m.','B-HOUSE_TYPE-Renov.','B-HOUSE_TYPE-Specpr.','B-HOUSE_TYPE-Staļina'])]=np.nan
correlation_table=correlation_table.drop(['Prices per day','Prices per month','Prices full','B-ESTATE_TYPE-farms-estates','B-ESTATE_TYPE-flats','B-ESTATE_TYPE-homes-summer-residences','B-ESTATE_TYPE-offices','B-ESTATE_TYPE-plots-and-lands','B-TRANSACTION_TYPE-hand-over','B-TRANSACTION_TYPE-sell'], axis=1)

summary_table=update_Index(summary_table)

summary_table.loc[(summary_table['Value of determinant']<0,'Change')]='a decrease'
summary_table.loc[(summary_table['Value of determinant']>0,'Change')]='an increase'
summary_table['name of determinant']=summary_table['Name of determinant'].str.lower()
summary_table[['n1','n2','n3','n4']] =summary_table['name of determinant'].str.split('-',expand=True)
summary_table.loc[(summary_table['n2']=='house_type'),'name of determinant']=summary_table['n3']
summary_table.loc[(summary_table['n2']=='district'),'name of determinant']=summary_table['n4']
summary_table.loc[(summary_table['n2']=='duration'),'name of determinant']=summary_table['n2']
summary_table.loc[(summary_table['n2']=='duration'),'duration1']=summary_table['n3']
summary_table.loc[(summary_table['n2']=='duration'),'duration2']=summary_table['n4']

summary_table.loc[(summary_table['Value of determinant']<1),'Value of determinant']=summary_table['Value of determinant'].round(4)
summary_table.loc[(summary_table['Value of determinant']>=1)&(summary_table['Value of determinant']<100),'Value of determinant']=summary_table['Value of determinant'].round(2)
summary_table.loc[(summary_table['Value of determinant']>=100),'Value of determinant']=summary_table['Value of determinant'].round(0)

summary_table['value of determinant']=summary_table['Value of determinant'].abs()
summary_table['value of determinant']=summary_table['value of determinant'].astype(str)

summary_table.loc[((summary_table['P>|t|']<0.05)&((summary_table['name of determinant']=='premise')|(summary_table['name of determinant']=='land')),'Regression analysis')]='- increasing the '+summary_table['name of determinant']+' by 1 m2 leads to '+summary_table['Change']+' in the price by '+summary_table['value of determinant']+' EUR'
summary_table.loc[((summary_table['P>|t|']<0.05)&(summary_table['name of determinant']=='pricepremise'),'Regression analysis')]='- increasing the price per square of premise by 1 EUR/m2 leads to '+summary_table['Change']+' in the price by '+summary_table['value of determinant']+' EUR'
summary_table.loc[((summary_table['P>|t|']<0.05)&(summary_table['name of determinant']=='priceland'),'Regression analysis')]='- increasing the price per square of land by 1 EUR/m2 leads to '+summary_table['Change']+' in the price by '+summary_table['value of determinant']+' EUR'
summary_table.loc[((summary_table['P>|t|']<0.05)&(summary_table['name of determinant']=='quarter'),'Regression analysis')]='- increasing the '+summary_table['Name of determinant']+' by 1 leads to '+summary_table['Change']+' in the price by '+summary_table['value of determinant']+' EUR'
summary_table.loc[((summary_table['P>|t|']<0.05)&((summary_table['name of determinant']=='number of rooms')|(summary_table['name of determinant']=='number of floors')|(summary_table['name of determinant']=='floor')|(summary_table['name of determinant']=='max floor')),'Regression analysis')]='- increasing the '+summary_table['name of determinant']+' by 1 leads to '+summary_table['Change']+' in the price by '+summary_table['value of determinant']+' EUR'
summary_table.loc[((summary_table['P>|t|']<0.05)&(summary_table['n2']=='district'),'Regression analysis')]='- if a district is '+summary_table['name of determinant']+', it leads to '+summary_table['Change']+' in the price by '+summary_table['value of determinant']+' EUR'
summary_table.loc[((summary_table['P>|t|']<0.05)&(summary_table['n2']=='house_type'),'Regression analysis')]='- if a house type is '+summary_table['name of determinant']+', it leads to '+summary_table['Change']+' in the price by '+summary_table['value of determinant']+' EUR'

summary_table.loc[((summary_table['P>|t|']<0.05)&(summary_table['n2']=='duration')&(summary_table['duration1']==0),'Regression analysis')]='- if a duration of advertisement at site is less than 1 day, it leads to '+summary_table['Change']+' in the price by '+summary_table['value of determinant']+' EUR'
summary_table.loc[((summary_table['P>|t|']<0.05)&(summary_table['n2']=='duration')&(summary_table['duration1']==1),'Regression analysis')]='- if a duration of advertisement at site is 1 day, it leads to '+summary_table['Change']+' in the price by '+summary_table['value of determinant']+' EUR'
summary_table.loc[((summary_table['P>|t|']<0.05)&(summary_table['n2']=='duration')&(summary_table['duration1']!=0)&(summary_table['duration1']!=1),'Regression analysis')]='- if a duration of advertisement at site is from '+summary_table['duration1']+' to '+summary_table['duration2']+' days, it leads to '+summary_table['Change']+' in the price by '+summary_table['value of determinant']+' EUR'

summary_table[['t','P>|t|','Value of determinant','Standard deviation','95% Confidence','interval']]=summary_table[['t','P>|t|','Value of determinant','Standard deviation','95% Confidence','interval']].round(2)
summary_table=summary_table.drop(['name of determinant','value of determinant','Change','n1','n2','n3','n4','duration1','duration2'],axis=1)

description_1='Determinants for the price in the regression are '
description_2='. Statistically significant with 95% determinants are described in regression analysis'
description_riga=', and districts'
description_house_types=', house types'
description_farms='the price are square of land, square of land, the price per square of premise, price per square of premise, number of floors, quarter, and duration of advertisements'
description_offices='the price per square of premise, square of premise, a floor, a max floor, quarter, and duration of advertisements'
description_residences='the price are square of land, square of land, the price per square of premise, price per square of premise, number of floors, quarter, and duration of advertisements'
description_plots_lands='the price per square of land, square of land, quarter, duration of advertisements, and districts'
description_flats='the price are square of premise, price per square of premise, number of rooms, quarter, and duration of advertisements'

summary_table.loc[((summary_table['Name of determinant']=='Intercept')&(summary_table['Estate type']=='farms')&(summary_table['Region']!='riga'),'Regression analysis')]=description_1+description_farms+description_2
summary_table.loc[((summary_table['Name of determinant']=='Intercept')&(summary_table['Estate type']=='offices')&(summary_table['Region']=='riga'),'Regression analysis')]=description_1+description_offices+description_riga+description_2
summary_table.loc[((summary_table['Name of determinant']=='Intercept')&(summary_table['Estate type']=='offices')&(summary_table['Region']!='riga'),'Regression analysis')]=description_1+description_offices+description_2
summary_table.loc[((summary_table['Name of determinant']=='Intercept')&(summary_table['Estate type']=='residences')&(summary_table['Region']=='riga'),'Regression analysis')]=description_1+description_residences+description_riga+description_2
summary_table.loc[((summary_table['Name of determinant']=='Intercept')&(summary_table['Estate type']=='residences')&(summary_table['Region']!='riga'),'Regression analysis')]=description_1+description_residences+description_2
summary_table.loc[((summary_table['Name of determinant']=='Intercept')&(summary_table['Estate type']=='plots and lands')&(summary_table['Region']=='riga'),'Regression analysis')]=description_1+description_plots_lands+description_riga+description_2
summary_table.loc[((summary_table['Name of determinant']=='Intercept')&(summary_table['Estate type']=='plots and lands')&(summary_table['Region']!='riga'),'Regression analysis')]=description_1+description_plots_lands+description_2
summary_table.loc[((summary_table['Name of determinant']=='Intercept')&(summary_table['Estate type']=='flats')&(summary_table['Region']=='riga')&(summary_table['Type of regression']=='without house types'),'Regression analysis')]=description_1+description_flats+description_riga+description_2
summary_table.loc[((summary_table['Name of determinant']=='Intercept')&(summary_table['Estate type']=='flats')&(summary_table['Region']=='riga')&(summary_table['Type of regression']=='with house types'),'Regression analysis')]=description_1+description_flats+description_house_types+description_riga+description_2
summary_table.loc[((summary_table['Name of determinant']=='Intercept')&(summary_table['Estate type']=='flats')&(summary_table['Region']!='riga')&(summary_table['Type of regression']=='without house types'),'Regression analysis')]=description_1+description_flats+description_2
summary_table.loc[((summary_table['Name of determinant']=='Intercept')&(summary_table['Estate type']=='flats')&(summary_table['Region']!='riga')&(summary_table['Type of regression']=='with house types'),'Regression analysis')]=description_1+description_flats+description_house_types+description_2

summary_table[['t', 'P>|t|']] = summary_table[['t','P>|t|']].fillna(value=0)

score_table=update_Index(score_table)
score_table.loc[(score_table['Regression score']>=0.9,'Reliability')]='The regression model is extremely reliable'
score_table.loc[((score_table['Regression score']>=0.75)&(score_table['Regression score']<0.9),'Reliability')]='The regression model is highly reliable'
score_table.loc[((score_table['Regression score']>=0.5)&(score_table['Regression score']<0.75),'Reliability')]='The regression model is reliable'
score_table.loc[(score_table['Regression score']<0.5,'Reliability')]='The regression model is not enough reliable'

conn = sqlite3.connect('database.db')
dataframe_table.to_sql('dataframe_table', conn, if_exists='replace', index = False)
correlation_table.to_sql('correlation_table', conn, if_exists='replace', index = False)
summary_table.to_sql('summary_table', conn, if_exists='replace', index = False)
score_table.to_sql('score_table', conn, if_exists='replace', index = False)